# Deep Learning MTH 5320 
## Homework 3
## 12/1/2023
## By Josias Moukpe

## Object Detection Model Assignment

### Dataset Prep and Preprocessing (20 points)
- Collect or create a dataset of images containing the objects you want to detect annotated with bounding boxes.
- Implement data preprocessing techniques, including:
  - Resizing images
  - Normalizing pixels
- Hint: You can find pre-annotated datasets for download on [Roboflow](https://github.com/roboflow/supervision).

### Model Architecture (30 points)
- Train a YOLOv8 model on your selected dataset using the public repository available at [Ultralytics](https://github.com/ultralytics/ultralytics).
- Monitor the training progress and use appropriate techniques to prevent overfitting.
- Conduct at least 3 tuning experiments.

### Evaluation (20 points)
- Evaluate your trained model's performance on a test dataset using mAP (mean Average Precision).
- Visualize the model's predictions on sample images.
- Analyze the model's performance, discussing any specific failure modes on your test dataset.

### BONUS (25 bonus points)
- Design and train a custom CNN-based object detection model using a self-designed architecture.
- Alternatively, compare several publicly available models on the same dataset.
- Consider implementing Gaussian YOLO V8 and comparing it against YOLO V8.

Please ensure that your code includes docstrings and type hints as requested, and be thorough in your analysis and experimentation.

### Setup and Imports

In [1]:
# Import libraries
import torch
from ultralytics import YOLO

In [2]:
# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using {device} device')
# get device specs
print(torch.cuda.get_device_name(0))

Using cuda device
NVIDIA GeForce RTX 3060


In [3]:
# test of yolo  v8
model = YOLO('yolov8n.pt')

In [4]:
# vid_path = "../data/test_footage.mp4"
# results = model(source=vid_path, show=True, conf=0.4, save=True)

### Training

In [5]:
ocr_dataset_path = 'D:/College/Fall2023/DeepLearning/yolo_ocr'
yaml_path = 'D:/College/Fall2023/DeepLearning/yolo_ocr/data.yaml'
# labels already in yolo v8 format

In [6]:
# Training.
results = model.train(
    data=yaml_path,
    imgsz=640,
    epochs=1000,
    batch=8,
    name='yolov8n_ocr')

New https://pypi.org/project/ultralytics/8.0.222 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.221 🚀 Python-3.9.18 torch-2.1.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3060, 12287MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=D:/College/Fall2023/DeepLearning/yolo_ocr/data.yaml, epochs=10, patience=50, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=yolov8n_ocr2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, show=False, sav

wandb: Currently logged in as: erud1t3. Use `wandb login --relogin` to force relogin


Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
AMP: checks passed ✅


train: Scanning D:\College\Fall2023\DeepLearning\yolo_ocr\train\labels.cache... 4245 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4245/4245 [00:00<?, ?it/s]

train: WARNING ⚠️ D:\College\Fall2023\DeepLearning\yolo_ocr\train\images\03c3dfc7e998b8562_jpg.rf.3d9d76c2437bcfb5cf433c0b6306d1ac.jpg: 1 duplicate labels removed
train: WARNING ⚠️ D:\College\Fall2023\DeepLearning\yolo_ocr\train\images\185d5dfa193c4ced_jpg.rf.b3d2ab34086b9e8f050b514cc22b4b9a.jpg: 1 duplicate labels removed
train: WARNING ⚠️ D:\College\Fall2023\DeepLearning\yolo_ocr\train\images\24c1fc0bf63da8153_jpg.rf.bd40f6e750b1a7990a9945aadddc1ef1.jpg: 1 duplicate labels removed
train: WARNING ⚠️ D:\College\Fall2023\DeepLearning\yolo_ocr\train\images\421ca0ce7f9329622_jpg.rf.4e25470bde67152fea6834e88e803708.jpg: 1 duplicate labels removed
train: WARNING ⚠️ D:\College\Fall2023\DeepLearning\yolo_ocr\train\images\4572990fd64bb6be3_jpg.rf.7ce23d5cd8b919a7832485c4815d97c8.jpg: 1 duplicate labels removed
train: WARNING ⚠️ D:\College\Fall2023\DeepLearning\yolo_ocr\train\images\492f6d607aae58c12_jpg.rf.44d8f6885cd8664eeff661809ad807ab.jpg: 1 duplicate labels removed
train: WARNING ⚠️ D:\Co

...

      10/10      1.56G     0.7105     0.6777      1.005         32        640: 100%|██████████| 531/531 [00:55<00:00,  9.56it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 77/77 [00:11<00:00,  6.48it/s]


                   all       1221       7890      0.938       0.88      0.934      0.736

10 epochs completed in 0.211 hours.
Optimizer stripped from runs\detect\yolov8n_ocr2\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\yolov8n_ocr2\weights\best.pt, 6.2MB

Validating runs\detect\yolov8n_ocr2\weights\best.pt...
Ultralytics YOLOv8.0.221 🚀 Python-3.9.18 torch-2.1.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3060, 12287MiB)
Model summary (fused): 168 layers, 3012668 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 77/77 [00:14<00:00,  5.27it/s]


                   all       1221       7890      0.936      0.881      0.934      0.736
                     0       1221        408      0.748      0.917      0.891      0.713
                     1       1221        386      0.846       0.91       0.93      0.654
                     2       1221        399      0.964      0.955      0.981      0.773
                     3       1221        369      0.966      0.962       0.99      0.787
                     4       1221        326       0.98      0.901      0.977       0.75
                     5       1221        383      0.959      0.963      0.988      0.791
                     6       1221        393      0.977      0.968      0.981      0.783
                     7       1221        343      0.963      0.965       0.99      0.746
                     8       1221        346      0.913      0.948      0.973      0.762
                     9       1221        324      0.984      0.976      0.992      0.794
                     

lr/pg0,▂▆█▇▇▆▅▃▂▁
lr/pg1,▂▆█▇▇▆▅▃▂▁
lr/pg2,▂▆█▇▇▆▅▃▂▁
metrics/mAP50(B),▁▆▇███████
metrics/mAP50-95(B),▁▆▇▇██████
metrics/precision(B),▁▅▇▇██████
metrics/recall(B),▁▅▇▇██████
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
train/box_loss,█▅▄▃▃▂▂▁▁▁


### Evaluation

### Custom Yolo G8 Architecture in Pytorch (NO UltraLytics here unfortunately)